In [6]:
import tflearn
import _pickle as pickle
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
import numpy as np

In [7]:
def get_proper_images(raw):
    raw_float = np.array(raw, dtype=float) 
    images = raw_float.reshape([-1, 3, 32, 32])
    images = images.transpose([0, 2, 3, 1])
    return images

def onehot_labels(labels):
    return np.eye(100)[labels]

def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding = 'latin1')
    fo.close()
    return dict

In [8]:
import sys
X = get_proper_images(unpickle('cifar-100-python/train')['data'])

utf-8


In [9]:
Y = onehot_labels(unpickle('cifar-100-python/train')['fine_labels'])
X_test = get_proper_images(unpickle('cifar-100-python/test')['data'])
Y_test = onehot_labels(unpickle('cifar-100-python/test')['fine_labels'])

In [10]:
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

In [11]:
# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=15.)
#img_aug.add_random_crop((28, 28))
#img_aug.add_random_blur(sigma_max=5.0)

Arguments
incoming: Tensor. Incoming 4-D Tensor.
nb_filter: int. The number of convolutional filters.
filter_size: int or list of int. Size of filters.
strides: 'intor list ofint`. Strides of conv operation. Default: [1 1 1 1].
padding: str from "same", "valid". Padding algo to use. Default: 'same'.
activation: str (name) or function (returning a Tensor) or None. Activation applied to this layer (see tflearn.activations). Default: 'linear'.
bias: bool. If True, a bias is used.
weights_init: str (name) or Tensor. Weights initialization. (see tflearn.initializations) Default: 'truncated_normal'.
bias_init: str (name) or Tensor. Bias initialization. (see tflearn.initializations) Default: 'zeros'.
regularizer: str (name) or Tensor. Add a regularizer to this layer weights (see tflearn.regularizers). Default: None.
weight_decay: float. Regularizer decay parameter. Default: 0.001.
trainable: bool. If True, weights will be trainable.
restore: bool. If True, this layer weights will be restored when loading a model.
reuse: bool. If True and 'scope' is provided, this layer variables will be reused (shared).
scope: str. Define this layer scope (optional). A scope can be used to share variables between layers. Note that scope will override name.
name: A name for this layer (optional). Default: 'Conv2D'.

In [12]:
# Convolutional network building
network = input_data(shape=[None, 32, 32, 3])
network = conv_2d(network, 32, 3, strides=1, padding='same', activation='relu', bias=True, 
                  bias_init='zeros', weights_init='uniform_scaling')
network = max_pool_2d(network, 2 , strides=None, padding='same')
network = conv_2d(network, 64, 3, strides=1, padding='same', activation='relu', bias=True, 
                  bias_init='zeros', weights_init='uniform_scaling')
network = conv_2d(network, 64, 3 , strides=1, padding='same', activation='relu', bias=True, 
                  bias_init='zeros', weights_init='uniform_scaling')
network = max_pool_2d(network, 2 , strides=None, padding='same')
network = fully_connected(network, 600, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 100, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit(X, Y, n_epoch=50, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=100 , run_id="withoutaugpre")

Training Step: 24999  | total loss: 22.81755 | time: 233.807s
| Adam | epoch: 050 | loss: 22.81755 - acc: 0.0090 -- iter: 49900/50000
Training Step: 25000  | total loss: 22.79233 | time: 248.622s
| Adam | epoch: 050 | loss: 22.79233 - acc: 0.0101 | val_loss: 22.79559 - val_acc: 0.0100 -- iter: 50000/50000
--
